<a href="https://colab.research.google.com/github/Gowrisan1987/NLP/blob/master/Youtube_comments_spam_collection_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import pandas as pd
#import spacy
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [40]:
yt_df=pd.read_csv("/content/drive/My Drive/Data/Youtube04-Eminem.csv")

In [41]:
yt_df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12rwfnyyrbsefonb232i5ehdxzkjzjs2,Lisa Wellas,NaN,+447935454150 lovely girl talk to me xxx﻿,1
1,z130wpnwwnyuetxcn23xf5k5ynmkdpjrj04,jason graham,2015-05-29T02:26:10.652000,I always end up coming back to this song<br />﻿,0
2,z13vsfqirtavjvu0t22ezrgzyorwxhpf3,Ajkal Khan,NaN,"my sister just received over 6,500 new <a rel=...",1
3,z12wjzc4eprnvja4304cgbbizuved35wxcs,Dakota Taylor,2015-05-29T02:13:07.810000,Cool﻿,0
4,z13xjfr42z3uxdz2223gx5rrzs3dt5hna,Jihad Naser,NaN,Hello I&#39;am from Palastine﻿,1


In [42]:
yt_df.isnull().values.any()

True

In [43]:
yt_df['CONTENT'].isnull().values.any()

False

In [44]:
yt_df['CLASS'].value_counts()

1    245
0    203
Name: CLASS, dtype: int64

In [45]:
yt_df.shape

(448, 5)

In [46]:
#nlp=spacy.load('en_core_web_sm')

In [13]:
#x=yt_df.drop('CLASS',axis=1)

In [47]:
y=yt_df['CLASS']

In [48]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(yt_df)):
    review = re.sub('[^a-zA-Z]', ' ', yt_df['CONTENT'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
#def clean_text(token):
  #if token.is_stop==False and token.is_punct==False and not token.is_space and token.lemma_!='-PRON-':
  #  return True
  #return False

#def lemma_text(text):
    #cleaned_text=[token.lemma_.lower() for token in nlp(text) if clean_text(token)]
    #return cleaned_text


# Cleaning through Spacy utilities
#def cleaned_text(data):
  #for i in range(0,1):
   # text = []
    #doc=nlp(data)
    #for j in range(0,len(doc)):
      print(j)
    #token=[word.lemma_.lower().strip() for word in doc if word.lemma_ != '-PRON-' and word.is_stop==False and word.is_punct==False]
    #token=[word for word in token if word.is_stop==False and token.is_punct==False]
    #token = ' '.join(token)
    #text.append(token)

  #return text
  
#test1=yt_df['CONTENT'][2]
#test2=cleaned_text(test1)


In [ ]:
#text = []

#for i in range(0,len(yt_df)):
  #doc=nlp(yt_df['CONTENT'][i])
  #token=[word.lemma_.lower().strip() for word in doc if word.is_stop==False and word.is_punct==False and word.is_alpha==True]
  #token = ' '.join(token)
  
  #text.append(token)


In [17]:
#text[8]

In [50]:
voc_size=5000

In [51]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [52]:
onehot_repr[0]

[20, 1281, 3595, 1941]

In [53]:
sent_ln=20
padded_corpus=pad_sequences(onehot_repr,maxlen=sent_ln,padding='pre')

In [54]:
padded_corpus[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,   20, 1281, 3595, 1941], dtype=int32)

# Using LSTM approach

In [55]:
model=Sequential()
model.add(Embedding(voc_size,40,input_length=sent_ln))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [56]:
x_final=np.array(padded_corpus)
y_final=np.array(y)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.20, random_state=42)

In [58]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
6/6 [==============================] - 1s 92ms/step - loss: 0.6865 - accuracy: 0.7151 - val_loss: 0.6771 - val_accuracy: 0.7556
Epoch 2/10
6/6 [==============================] - 0s 63ms/step - loss: 0.6555 - accuracy: 0.7570 - val_loss: 0.6317 - val_accuracy: 0.6778
Epoch 3/10
6/6 [==============================] - 0s 32ms/step - loss: 0.5755 - accuracy: 0.7011 - val_loss: 0.5785 - val_accuracy: 0.6667
Epoch 4/10
6/6 [==============================] - 0s 30ms/step - loss: 0.5142 - accuracy: 0.7346 - val_loss: 0.5372 - val_accuracy: 0.7444
Epoch 5/10
6/6 [==============================] - 0s 31ms/step - loss: 0.4686 - accuracy: 0.7654 - val_loss: 0.5186 - val_accuracy: 0.7222
Epoch 6/10
6/6 [==============================] - 0s 32ms/step - loss: 0.4398 - accuracy: 0.7765 - val_loss: 0.4874 - val_accuracy: 0.7667
Epoch 7/10
6/6 [==============================] - 0s 30ms/step - loss: 0.4023 - accuracy: 0.8073 - val_loss: 0.4611 - val_accuracy: 0.7778
Epoch 8/10
6/6 [===========

In [59]:
y_pred=model.predict_classes(X_test)

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
confusion_matrix(y_test,y_pred)

array([[40,  3],
       [ 5, 42]])

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9111111111111111

## Trying using Bidirectional LSTM

In [63]:
model1=Sequential()
model1.add(Embedding(voc_size,40,input_length=sent_ln))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
6/6 [==============================] - 1s 157ms/step - loss: 0.6882 - accuracy: 0.6145 - val_loss: 0.6777 - val_accuracy: 0.7444
Epoch 2/10
6/6 [==============================] - 0s 41ms/step - loss: 0.6609 - accuracy: 0.7151 - val_loss: 0.6367 - val_accuracy: 0.6667
Epoch 3/10
6/6 [==============================] - 0s 42ms/step - loss: 0.5822 - accuracy: 0.7123 - val_loss: 0.5265 - val_accuracy: 0.7333
Epoch 4/10
6/6 [==============================] - 0s 43ms/step - loss: 0.4453 - accuracy: 0.7989 - val_loss: 0.3836 - val_accuracy: 0.8556
Epoch 5/10
6/6 [==============================] - 0s 43ms/step - loss: 0.2932 - accuracy: 0.8687 - val_loss: 0.3737 - val_accuracy: 0.8556
Epoch 6/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1673 - accuracy: 0.9385 - val_loss: 0.2852 - val_accuracy: 0.9222
Epoch 7/10
6/6 [==============================] - 0s 43ms/step - loss: 0.0667 - accuracy: 0.9777 - val_loss: 0.3273 - val_accuracy: 0.9111
Epoch 8/10
6/6 [==========

In [65]:
y_pred_model1=model1.predict_classes(X_test)

In [66]:
confusion_matrix(y_test,y_pred_model1)

array([[37,  6],
       [ 3, 44]])

In [67]:
accuracy_score(y_test,y_pred_model1)

0.9